# pepStream notebook
This code relies on packages : \n

To run this using cluster resources please run : \n

Copy all the run files to a new directory to prevent overwriting output \n

In [1]:
# Define the search sequence here : Example is xrd of RIN4 - CNOI of RIN4
MySequence = "PKPTNLRADESPEKVTVVPKFGDWDENNPSSADGYTHIFNKVREERSSGA" # Please keep the "" around the sequence


with open('sequenceOfInterest.fasta', 'w') as f:
    print(">contigous sequence for pepStream auto-designed peptides", file=f)
    print(MySequence, file=f)
f.close()

# Creates the search fragements of the seqeuence

In [2]:
# Variable c1 defines how many partitions to make. Note: at least 3 letter long sequences are needed for BLAST
#7
%run WindowFootprint.py --f1 sequenceOfInterest.fasta --c1 13 

ModuleNotFoundError: No module named 'Bio'

In [3]:
pwd

'/users/tgrear/pepStream'

In [4]:
# get new dir name with fragmented sequences
#os.system("ls")
import os
os.listdir()

['mutated_footprints2.0.csv',
 'Footprint_6',
 'mutated_footprints3.0.fasta',
 'Footprint_3',
 'mutantFootprints_2.py',
 'mutated_footprints1.0.csv',
 'error_DivSeqs.err',
 'shortblast.log',
 'Footprint_1',
 'make_mutatnt_seqs.sh',
 'mutated_footprints4.0.csv',
 'DivSeqs.log',
 'mutated_footprints3.0.csv',
 'error_BLAST.err',
 'mutated_footprints2.0.fasta',
 'Footprint_2',
 'Footprint_5',
 'PSS2Seqs.py',
 'run_psiBLAST.slurm',
 'mutated_footprints1.0.fasta',
 'Footprint_0',
 'Footprint_4',
 'mutated_footprints4.0.fasta']

In [4]:
# navigate into said directory: might be unneeded if windowfootprint was run 
#os.chdir('..')
os.chdir('out_seq_934632')

FileNotFoundError: [Errno 2] No such file or directory: 'out_seq_934632'

In [32]:
# Submit psiBLAST job, first BLAST analysis
## MUST move run scipt into dir ##
os.system("cp ../run_psiBLAST.slurm .")
os.system("sbatch run_psiBLAST.slurm")

0

In [2]:
# The previous jobs must finish entirely before running the next step
# To see status of your jobs on slurm cluster
import os
print(os.popen('squeue -u jpatte75').read())


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1755564     Orion pBLAST_a jpatte75  R    4:36:35      1 str-c141



In [46]:
# Create mutant sequences for searching PDB
os.system("cp ../make_mutatnt_seqs.sh .")
os.system("cp ../mutantFootprints_2.py .")
os.system("sbatch make_mutatnt_seqs.sh")
#os.system("./make_mutatnt_seqs.sh")
#print(os.popen("./make_mutatnt_seqs.sh").read())
print(os.popen('squeue -u jpatte75').read())
os.listdir()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1754950     Orion  DivSeqs jpatte75 PD       0:00      1 (Priority)



['mutated_footprints2.0.csv',
 'Footprint_6',
 'mutated_footprints3.0.fasta',
 'Footprint_3',
 'mutantFootprints_2.py',
 'mutated_footprints1.0.csv',
 'error_DivSeqs.err',
 'shortblast.log',
 'Footprint_1',
 'make_mutatnt_seqs.sh',
 'mutated_footprints4.0.csv',
 'mutated_footprints.fasta',
 'DivSeqs.log',
 'fragSO.py',
 'make_specific_docks.sh',
 'mutated_footprints3.0.csv',
 'error_BLAST.err',
 'homologs.fasta',
 'mutated_footprints2.0.fasta',
 'Footprint_2',
 'results.out',
 'Footprint_5',
 'PSS2Seqs.py',
 'run_psiBLAST.slurm',
 'mutated_footprints1.0.fasta',
 'Footprint_0',
 'getFragSO_v2.py',
 'Footprint_4',
 'mutated_footprints4.0.fasta',
 'run_pBLAST.slurm']

In [47]:
# The previous jobs must finish entirely before running the next step
# To see status of your jobs on slurm cluster
print(os.popen('squeue -u jpatte75').read())
print(os.popen('cat error_DivSeqs.err').read())
print(os.popen('tail -5 DivSeqs.log').read())


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1754950     Orion  DivSeqs jpatte75 PD       0:00      1 (Priority)

Smith-Waterman local alignment of sequences
Smith-Waterman local alignment of sequences
slurmstepd: error: *** JOB 1754948 ON str-c149 CANCELLED AT 2021-12-13T15:29:32 ***
slurmstepd: error: *** JOB 1754949 ON str-c53 CANCELLED AT 2021-12-13T15:29:32 ***




In [49]:
# Search PDB for distant strucutral hits, then mine these matches for peptides
# This will produce raw fasta reads for the peptides, this will take a significant amount of time
os.system("cp ../run_pBLAST.slurm .")
os.system("sbatch run_pBLAST.slurm")

0

In [84]:
# The previous jobs must finish entirely before running the next step
# To see status of your jobs on slurm cluster
print(os.popen('squeue -u jpatte75').read())
print(os.popen('cat error_BLAST.err').read())


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1754963     Orion pBLAST_a jpatte75  R    6:56:50      1 str-c149
           1754983     Orion dockmode jpatte75  R    6:32:20      1 str-c137

cat: mutated_footprints.fasta: input file is output file



In [66]:
# You MUST create a docking model of the entire target protein
# This can be run immediatly after the previous step as it is an unrelated computation

# Define the target protein sequence here : Example is RIN4 
MySequence = "MARSNVPKFGNWEAEENVPYTAYFDKARKTRAPGSKIMNPNDPEYNSDSQSQAPPHPPSSRTKPEQVDTV\
RRSREHMRSREESELKQFGDAGGSSNEAANKRQGRASQNNSYDNKSPLHKNSYDGTGKSRPKPTNLRADESPEKVTVVPKFGDWDENNPS\
SADGYTHIFNKVREERSSGANVSGSSRTPTHQSSRNPNNTSSCCCFGFGGK" 
# Please keep the "" around the sequence


with open('proteinOfInterest_RIN4.fasta', 'w') as f:
    print(">protein sequence for pepStream docking routine", file=f)
    print(MySequence, file=f)
f.close()

os.system("cp ../run_iTasser_dockmodel.sh .")
os.system("sbatch run_iTasser_dockmodel.sh")

0

In [ ]:
# After generating valid cMoRFs and the docking strucutres
# Run I-TASSER  Z-DOCK : pass 1
print(os.popen('sbatch runPepStream_PhysicalFilter.sh').read())

In [ ]:
# The previous jobs must finish entirely before running the next step
# There will be many many jobs
# To see status of your jobs on slurm cluster
print(os.popen('squeue -u jpatte75').read())

In [ ]:
# This creates a corse grain sort order off overall afffinity of the designed peptide against the entire target structure
%run format_output.py 

In [ ]:
# Run Z-DOCK : pass 2
# Define the specific sites of docking
import numpy

MoRF1=numpy.arange(10, 30, 1)
        numpy.savetxt('MoRF1_Block_Residues.txt', MoRF1, delimiter =', ')
MoRF12=10:1:50
        numpy.savetxt('MoRF12_Block_Residues.txt',MoRF12, delimiter =', ')
MoRF2=40:1:50
        numpy.savetxt('MoRF2_Block_Residues.txt', MoRF2, delimiter =', ')
MoRF3=140:1:160
        numpy.savetxt('MoRF3_Block_Residues.txt', MoRF3, delimiter =', ')
Loop=100:1:120
        numpy.savetxt('Loop_Block_Residues.txt', Loop, delimiter =', ')

os.system("./make_specific_docks.sh")
# Fine grain sort order by docking specific subsites of the entire target strucutre
# False positives are included in this evaluation to understand specificity of the binding peptide
# This must be defined in the docking scripts for new target struct
os.system("sbatch runPepStream_AnalyzeFilter.slurm")

In [ ]:
# The previous jobs must finish entirely before running the next step
# To see status of your jobs on slurm cluster
print(os.popen('squeue -u jpatte75').read())
print(os.popen('cat error_BLAST.err').read())

In [ ]:
# Work up results
# Contains graphical outputs, textual outputs for sorted predictions, and runs ML models over the data to predict best sort
# Note that the ML routines are trained off DELLA peptide data 
%run format_output_specific.py